In [1]:
import nest_asyncio
nest_asyncio.apply()

In [2]:
import stan
import arviz as az
import numpy as np

In [3]:
schools_code = """
data {
    int<lower=0> J;
    real y[J];
    real<lower=0> sigma[J];
}

parameters {
    real mu;
    real<lower=0> tau;
    real theta_tilde[J];
}

transformed parameters {
    real theta[J];
    for (j in 1:J)
        theta[j] = mu + tau * theta_tilde[j];
}

model {
    mu ~ normal(0, 5);
    tau ~ cauchy(0, 5);
    theta_tilde ~ normal(0, 1);
    y ~ normal(theta, sigma);
}

generated quantities {
    vector[J] log_lik;
    vector[J] y_hat;
    for (j in 1:J) {
        log_lik[j] = normal_lpdf(y[j] | theta[j], sigma[j]);
        y_hat[j] = normal_rng(theta[j], sigma[j]);
    }
}
"""

eight_school_data = {
    "J": 8,
    "y": np.array([28.0, 8.0, -3.0, 7.0, -1.0, 1.0, 18.0, 12.0]),
    "sigma": np.array([15.0, 10.0, 16.0, 11.0, 9.0, 11.0, 10.0, 18.0]),
}

In [5]:
posterior = stan.build(schools_code,data=eight_school_data)

Building...

In file included from /home/mike/venvs/mmm_venv/lib/python3.9/site-packages/httpstan/include/stan/math/prim/fun.hpp:110,
                 from /home/mike/venvs/mmm_venv/lib/python3.9/site-packages/httpstan/include/stan/math/rev/fun.hpp:7,
                 from /home/mike/venvs/mmm_venv/lib/python3.9/site-packages/httpstan/include/stan/math/rev.hpp:10,
                 from /home/mike/venvs/mmm_venv/lib/python3.9/site-packages/httpstan/include/stan/math.hpp:19,
                 from /home/mike/venvs/mmm_venv/lib/python3.9/site-packages/httpstan/include/stan/model/model_header.hpp:4,
                 from /root/.cache/httpstan/4.5.0/models/nnrawek7/model_nnrawek7.cpp:2:
/home/mike/venvs/mmm_venv/lib/python3.9/site-packages/httpstan/include/stan/math/prim/fun/generalized_inverse.hpp: In function ‘Eigen::Matrix<typename stan::value_type<T>::type, EigMat::ColsAtCompileTime, EigMat::RowsAtCompileTime> stan::math::generalized_inverse(const EigMat&)’:
/home/mike/venvs/mmm_venv/lib/python3.9/si


Building: 42.7s, done.

In [7]:
fit = posterior.sample(num_chains=4, num_samples=1000)

Sampling:   0%
Sampling:  25% (2000/8000)
Sampling:  50% (4000/8000)
Sampling:  75% (6000/8000)
Sampling: 100% (8000/8000)
Sampling: 100% (8000/8000), done.
Messages received during sampling:
  Gradient evaluation took 2.6e-05 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.26 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 8e-06 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.08 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 1.3e-05 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.13 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 1.5e-05 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.15 seconds.
  Adjust your expectations accordingly!


In [22]:
stan_data = az.from_pystan(
    posterior=fit,
    posterior_predictive="y_hat",
    observed_data=["y"],
    log_likelihood={"y": "log_lik"},
    coords={"school": np.arange(eight_school_data["J"])},
    dims={
        "theta": ["school"],
        "y": ["school"],
        "log_lik": ["school"],
        "y_hat": ["school"],
        "theta_tilde": ["school"],
    },
    posterior_model=posterior,
)

In [28]:
stan_data

Inference data with groups:
	> posterior
	> posterior_predictive
	> log_likelihood
	> sample_stats
	> observed_data

In [10]:
import matplotlib.pyplot as plt

In [19]:
dir(stan_data)

['InferenceDataItemsView',
 'InferenceDataValuesView',
 '__abstractmethods__',
 '__add__',
 '__class__',
 '__class_getitem__',
 '__contains__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__orig_bases__',
 '__parameters__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__reversed__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_group_names',
 '_groups',
 '_groups_all',
 '_groups_warmup',
 '_is_protocol',
 '_repr_html_',
 '_wrap_xarray_method',
 'add_groups',
 'assign',
 'assign_coords',
 'chunk',
 'compute',
 'copy',
 'cumsum',
 'extend',
 'from_netcdf',
 'from_zarr',
 'get',
 'get_index',
 'groups',
 'isel',
 'items',
 'keys',
 'load',
 'log_likelihood',
 'map',
 'max',
 'mean',
 'median',
 'm

In [20]:
data = az.load_arviz_data("non_centered_eight")

In [21]:
data

Inference data with groups:
	> posterior
	> posterior_predictive
	> sample_stats
	> prior
	> observed_data